In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import os

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# make np image 
def makenp_image(image_directory):
    SIZE = 224
    dataset = []
    my_images = os.listdir(image_directory)
    for _, image_name in tqdm(enumerate(my_images), total=len(my_images)): 
        if (image_name.endswith(('jpg', 'png', 'JPG', 'PNG'))):    
            # print(image_directory + image_name)
            image = io.imread(image_directory + image_name)        
            image = Image.fromarray(image, 'RGB')        
            image = image.resize((SIZE,SIZE)) 
            dataset.append(np.array(image))
    return np.array(dataset)

# gen augment image
def genimg_augmentation(datagen, x, LABEL, DIRECTORY):
    i = 0
    for batch in datagen.flow(x, batch_size=len(x),
                              save_to_dir= DIRECTORY,
                              save_prefix='augment-' + LABEL,
                              save_format='jpg'
                              ):
        i += 1
        if i > 0:
            break

In [4]:
## monitoring parameter 
LABEL = ["cbb", "cbsd", "cgm", "cmd", "healthy"]
option = 'load' ##create or load

# Define the augmentationss
augmentations = [
    (ImageDataGenerator(brightness_range=[0.6, 0.9]), 'brightness'),
    #(ImageDataGenerator(horizontal_flip=True), 'horizontal_flip'),
    #(ImageDataGenerator(zoom_range=[0.6, 0.9]), 'zoom'),
    #(ImageDataGenerator(vertical_flip=True), 'vertical_flip'),
    #(ImageDataGenerator(rotation_range=40), 'rotation')
]

In [6]:
classnp = './np/'
os.makedirs(classnp, exist_ok=True)

In [5]:
for label in LABEL:
    DIRECTORY = r'dataset/original/' + label + '/'

    print(f"class {label} :")
    if option == 'create':
        # Generate a numpy array representation of an image for a given label
        x = makenp_image(DIRECTORY)

        # Save the numpy array as a .npy file with the same name as the label
        np.save(f'{classnp}/{label}.npy', x)
    else:
        # Load the numpy array from the .npy file
        x = np.load(f'{classnp}/{label}.npy')

    # Apply augmentations
    for augmentation, method in augmentations:
        genimg_augmentation(augmentation, x, label, DIRECTORY)
print('finish...')

100%|██████████| 1812/1812 [00:35<00:00, 51.76it/s]
